In [35]:
import numpy as np
import matplotlib.pyplot as plt
import numexpr as ne
import scipy.signal as signal
%matplotlib qt5

In [113]:
rampsPerFile = 1500
samplesPerRamp = 6000
samplesPerUpRamp = 3000
nbFiles = 10

fmin = 5.725e9
fMAX = 5.875e9
fc = 5.8e9
B = fMAX - fmin
T = 600e-6
alpha = (fMAX - fmin) / T
c = 3e8
kc = 4 * np.pi * fc / c

V = 40

print( "kc = {:.2f}, B = {}".format(kc, B) )

kc = 242.95, B = 150000000.0


In [193]:
Naz = nbFiles * rampsPerFile
xa_vec = np.arange( Naz ) * T * V

### Image geometry

In [183]:
x_min = 150
x_max = 200
r_min = 50
r_max = 150

#extent=[horizontal_min,horizontal_max,vertical_min,vertical_max]
extent=[r_min,r_max,x_max,x_min]

d_x = 0.5
d_r = 0.5

x = np.arange( x_min, x_max, d_x )
x = x.reshape( x.size, 1 )
r = np.arange( r_min, r_max, d_r )
r = r.reshape( 1, r.size )

print( "img.shape = {}, x.shape = {}, r.shape = {}".format( img.shape, x.shape, r.shape ) )

img.shape = (100, 200), x.shape = (100, 1), r.shape = (1, 200)


In [195]:
overSamplingRatio = 4
nbPointsResampled = overSamplingRatio * srf.shape[1]
rangeResolution = c / (2 * B)
r_oversampled = np.arange( 0, nbPointsResampled ) * rangeResolution / overSamplingRatio
print( "rangeResolution = {}, {:.2f} to {:.2f}".format( rangeResolution, r_overSampled[0], r_overSampled[-1] ) )

rangeResolution = 1.0, 0.00 to 2999.75


### Read samples

In [175]:
data_dir = "/home/pleroy/DATA/SIMU"

In [186]:
firstFile = 0
lastFile = 9
adc = np.load(data_dir + '/adc_files_{}_{}.npy'.format(firstFile, lastFile) ) 
print( "adc.shape = {}".format( adc.shape ) )

adc.shape = (15000, 3000)


In [187]:
srf = np.fft.ifft( adc, axis=1 )

### Backprojection

In [191]:
nbLoops = positions.size
loop = 0
img  = np.zeros( (x.size, r.size), dtype=complex )
for xa in xa_vec:
    if loop%1000 == 0:
        print( "{} / {}".format(loop, nbLoops ) )
    img += np.exp( 1j * kc * (r**2 + (xa-x)**2 )**0.5 ) \
            * np.interp( (r**2 + (xa-x)**2 )**0.5, r_oversampled, signal.resample( srf[loop,:], nbPointsResampled  ) )
    loop += 1
# np.exp( -1j * kc * r) \

0 / 15000
1000 / 15000
2000 / 15000
3000 / 15000
4000 / 15000
5000 / 15000
6000 / 15000
7000 / 15000
8000 / 15000
9000 / 15000
10000 / 15000
11000 / 15000
12000 / 15000
13000 / 15000
14000 / 15000


In [206]:
plt.figure()
plt.imshow( ((r**2 + (175-x)**2 )**0.5) )
plt.colorbar()

In [209]:
plt.figure()
plt.plot( ((r**2 + (175-x)**2 )**0.5)[:,0] )
plt.grid()

In [192]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( img ) ), extent=extent )
plt.grid()
plt.colorbar()

### Resample

In [189]:
plt.imshow( 20 * np.log10( np.abs( srf[::10, :] ) ) )

In [85]:
nbPointsResampled = 4 * srf.shape[1]
srf_resampled = signal.resample( srf[0,:], nbPointsResampled  )
print( "srf.shape = {}, srf_resamples.shape = {}".format( srf.shape, srf_resampled.shape ) )

srf.shape = (15000, 3000), srf_resamples.shape = (12000,)


In [105]:
up_vec  = np.zeros( nbPointsResampled, dtype = complex )
vec_RD  = np.zeros( nbPointsResampled, dtype = complex  )

Nf = srf.shape[1]

vec_ind = int( np.ceil( ( Nf + 1 ) / 2 ) )

pos = 0
#upsample raw data
vec_RD[ 0 : vec_ind ] = adc[ pos, 0 : vec_ind ]
vec_RD[ - ( Nf - vec_ind ) : ] = adc[ pos, vec_ind : ]
# vec_RD contains RD(pos,:) and zeros: up-sampling

up_vec = np.fft.ifft( vec_RD ) # Transformation to time domain

print( "adc.shape = {}, vec_ind = {}, vec_RD.shape = {}".format( adc.shape, vec_ind, vec_RD.shape ) )

adc.shape = (15000, 3000), vec_ind = 1501, vec_RD.shape = (12000,)


In [111]:
plt.figure()
plt.subplot(211)
plt.plot( np.arange( srf.shape[1] ), np.real( srf[0,:] ), 'go-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.real( srf_resampled ), 'b.-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.real( up_vec ) * 4, '.-', color="pink" )
ax = plt.gca()
plt.subplot(212, sharex=ax)
plt.plot( np.arange( srf.shape[1] ), np.imag( srf[0,:] ), 'go-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.imag( srf_resampled ), 'b.-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.imag( up_vec ) * 4, '.-', color="pink" )